1. Import the standard libraries and others as needed.

In [377]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
import datetime

2. Connect to MongoDB.
3. Restore the database from the dump given in the ‘data.zip’ file. This database contains two collections: ‘patients’ and ‘conditions’.

In [729]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["patientdb"]

# Restore
# mongorestore --db patientdb --collection patients --drop patients.bson
# mongorestore --db patientdb --collection conditions --drop conditions.bson

4. Import the ‘patient_data_1.csv’ and ‘patient_data_2.csv’ into pandas DataFrames and harmonize them. Note the differences in Ethnicity and Gender fields mentioned in the instructions above. Also, in both DataFrames, change the name of the column indicating patient id to ‘_id’. [30 points]

In [383]:
# Load data
df1 = pd.read_csv("patient_data_1.csv")
df2 = pd.read_csv("patient_data_2.csv")

print(df1.head())
print(df2.head())

        Id   BIRTHDATE MARITAL   RACE    ETHNICITY  GENDER       CITY  \
0  P004753  2012-01-12     NaN  asian     hispanic  Female   Westford   
1  P009495  1961-12-01       M  white  nonhispanic  Female    Seekonk   
2  P010851  1997-10-21     NaN  black  nonhispanic    Male     Quincy   
3  P009020  1975-08-15       M  white  nonhispanic  Female  Worcester   
4  P009779  2020-02-25     NaN  white  nonhispanic  Female     Bolton   

             COUNTY          STATE  
0  Middlesex County  Massachusetts  
1    Bristol County  Massachusetts  
2    Norfolk County  Massachusetts  
3  Worcester County  Massachusetts  
4  Worcester County  Massachusetts  
        id   birthdate marital   race     ethnicity gender             city  \
0  P005102  1969-05-11       M  white  non-hispanic      F  East Longmeadow   
1  P001814  1957-12-31       M  white  non-hispanic      M       Somerville   
2  P000373  1989-07-20       S  white  non-hispanic      F        Cambridge   
3  P007365  1967-10-26 

In [385]:
# Convert column names to uppercase
df1.columns = df1.columns.str.upper()
df2.columns = df2.columns.str.upper()

# Standardize gender values
df1["GENDER"] = df1["GENDER"].replace({"MALE": "M", "FEMALE": "F"})

# Standardize ethnicity values
df2["ETHNICITY"] = df2["ETHNICITY"].replace({"non-hispanic": "nonhispanic"})

# Change the name of the column indicating patient id to ‘_id’
df1.rename(columns={"ID": "_id"}, inplace=True)
df2.rename(columns={"ID": "_id"}, inplace=True)

print(df1.head())
print(df2.head())

       _id   BIRTHDATE MARITAL   RACE    ETHNICITY  GENDER       CITY  \
0  P004753  2012-01-12     NaN  asian     hispanic  Female   Westford   
1  P009495  1961-12-01       M  white  nonhispanic  Female    Seekonk   
2  P010851  1997-10-21     NaN  black  nonhispanic    Male     Quincy   
3  P009020  1975-08-15       M  white  nonhispanic  Female  Worcester   
4  P009779  2020-02-25     NaN  white  nonhispanic  Female     Bolton   

             COUNTY          STATE  
0  Middlesex County  Massachusetts  
1    Bristol County  Massachusetts  
2    Norfolk County  Massachusetts  
3  Worcester County  Massachusetts  
4  Worcester County  Massachusetts  
       _id   BIRTHDATE MARITAL   RACE    ETHNICITY GENDER             CITY  \
0  P005102  1969-05-11       M  white  nonhispanic      F  East Longmeadow   
1  P001814  1957-12-31       M  white  nonhispanic      M       Somerville   
2  P000373  1989-07-20       S  white  nonhispanic      F        Cambridge   
3  P007365  1967-10-26     

5. From the ‘patients’ collection, retrieve the patients who are alive. Print the number of patients retrieved. This should be 8,700. Store them in a DataFrame. [20 points]

In [388]:
alive_patients = list(db.patients.find(
    {"DEATHDATE": None},  # Filter alive patients
    {"_id": 1, "BIRTHDATE": 1, "GENDER": 1, "ETHNICITY": 1}
))

# Convert to DataFrame
alive_df = pd.DataFrame(alive_patients)

# Print the number of patients retrieved
print(len(alive_df))

8700


6. Concatenate the above three DataFrames. You should get 10,000 patients. It should be as follows when sorted by the ‘_id’ field. [10 points]

In [391]:
combined_df = pd.concat([df1, df2, alive_df], ignore_index=True)
combined_df.sort_values(by="_id", inplace=True)

print(len(combined_df))

10000


7. Compute the current age of the patients and store them in another column called ‘AGE’. Note that you may need to convert the data type of the ‘BIRTHDATE’ field to achieve this. [30 points]

In [394]:
from datetime import datetime

# Convert birthdate to datetime
combined_df["BIRTHDATE"] = pd.to_datetime(combined_df["BIRTHDATE"])

# Compute current age
combined_df["AGE"] = combined_df["BIRTHDATE"].apply(lambda x: datetime.now().year - x.year)

# Display first few rows
print(combined_df[["BIRTHDATE", "AGE"]].head())

      BIRTHDATE  AGE
1300 2017-08-24    8
1301 2016-08-01    9
1302 1992-06-30   33
1303 2004-01-09   21
1304 1996-11-15   29


8. Insert the above DataFrame to a collection named ‘patients_clean’. Display the number of documents in the collection to make sure that all the records have been inserted properly. [10 points]

In [397]:
db.patients_clean.drop()
db.patients_clean.insert_many(combined_df.to_dict("records"))

# Display the number of documents
print(db.patients_clean.count_documents({}))

10000


9. From the conditions collection, obtain the ID’s of patients that were diagnosed with COVID-19. You should get 8,820 patients here (without filtering out deceased patients). [10 points]

In [400]:
covid_patients = db.conditions.distinct("patient", {"conditions": {"$in": ["COVID-19"]}})
print(len(covid_patients))

8820


10. From the conditions collection, create an aggregation pipeline to obtain COVID-19 patients who also had ‘Cough (finding)’ and ‘Fever (finding)’ but did not have ‘Loss of taste (finding)’. Note that here you need to consider the existence/non-existence of these conditions in the same visit. You should get 2,613 patients here (without filtering out deceased patients). [20 points]

In [403]:
pipeline_10 = list(db.conditions.aggregate([
    {
        "$match": {
            "conditions": {
                "$all": ["COVID-19", "Cough (finding)", "Fever (finding)"],
                "$nin": ["Loss of taste (finding)"]
            }
        }
    }
]))

print(len(pipeline_10))

2613


11. Create a new aggregation pipeline to join patient data for 8,820 COVID-19 patients. Remove deceased patients here. Note that the ‘patients_clean’ collection only contains alive patients while ‘conditions collection contains data for both alive and the deceased. You should have data for 8,463 patients here. [40 points]

In [406]:
pipeline_11 = [
    {"$match": {"conditions": "COVID-19"}},
    {"$lookup": {
        "from": "patients_clean", 
        "localField": "patient", 
        "foreignField": "_id", 
        "as": "patient_info"
    }},
    {"$match": {"patient_info": {"$ne": []}}},
    {"$group": {"_id": "$patient"}}
]

result_11 = list(db.conditions.aggregate(pipeline_11))
print(len(result_11))

8463


12. Based on the previous pipeline, create a new pipeline to calculate the total number of alive COVID-19 patients for each race. Display the results. You should get 7089 "white", 709 "black", 610 "asian", 47 "native", and 8 "other" patients. [30 points]

In [696]:
db.patients_clean.distinct("RACE") # Race values in 'patients_clean'

['asian', 'black', 'native', 'other', 'white']

In [628]:
pipeline_12 = list(db['patients_clean'].aggregate([
    {"$lookup": {
        "from": "conditions",
        "localField": "_id",
        "foreignField": "patient",
        "as": "patient_conditions"
    }},
    {"$match": {
        "patient_conditions.conditions": {"$in": ["COVID-19"]}
    }},
    {"$group": {
        "_id": "$RACE",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]))

print(pipeline_12)

[{'_id': 'white', 'count': 6442}, {'_id': 'black', 'count': 656}, {'_id': 'asian', 'count': 548}, {'_id': 'native', 'count': 40}, {'_id': 'other', 'count': 7}]


In [644]:
print(db.patients_clean.count_documents({})) # Verify total number of patients

11052


In [646]:
print(db.conditions.count_documents({"conditions": "COVID-19"})) # Verifyu COVID-19 patient count

8820


In [640]:
print(db.patients_clean.count_documents({"RACE": "other"})) # Verify "other" count

8


In [690]:
pipeline_12 = list(db['patients_clean'].aggregate([
    {"$lookup": {
        "from": "conditions",
        "localField": "_id",
        "foreignField": "patient",
        "as": "patient_conditions"
    }},
    {"$match": {
        "patient_conditions.conditions": {"$in": ["COVID-19"]}
    }},
    {"$group": {
        "_id": "$RACE",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]))

print(pipeline_12)

[{'_id': 'white', 'count': 6442}, {'_id': 'black', 'count': 656}, {'_id': 'asian', 'count': 548}, {'_id': 'native', 'count': 40}, {'_id': 'other', 'count': 7}]


In [702]:
covid_patient_ids = set(db.conditions.distinct("patient", {"conditions": "COVID-19"}))
existing_patient_ids = set(db.patients_clean.distinct("_id"))

print (len(covid_patient_ids))
print(len(existing_patient_ids))

8820
11052


In [704]:
missing_covid_patients = covid_patient_ids - existing_patient_ids

print(len(missing_covid_patients))

1127


In [706]:
missing_patient_data = list(db.patients.find({"_id": {"$in": list(missing_patients)}})) # Restore missinng patients

In [714]:
sample_missing_ids = list(missing_patients)[:10]

existing_missing_patients = list(db.patients.find({"_id": {"$in": sample_missing_ids}}))

print(len(existing_missing_patients))

0


In [716]:
patient_sample = db.patients.find_one() # Check ID format in 'patients'
print("Sample patient record:", patient_sample)

Sample patient record: {'_id': 'P000001', 'BIRTHDATE': '2017-08-24', 'MARITAL': nan, 'RACE': 'white', 'ETHNICITY': 'nonhispanic', 'GENDER': 'M', 'CITY': 'Springfield', 'COUNTY': 'Hampden County', 'STATE': 'Massachusetts'}


In [718]:
clean_sample = db.patients_clean.find_one() # Check ID format in 'patients_clean'
print("Sample clean patient record:", clean_sample)

Sample clean patient record: {'_id': 'P000001', 'BIRTHDATE': '2017-08-24', 'MARITAL': nan, 'RACE': 'white', 'ETHNICITY': 'nonhispanic', 'GENDER': 'M', 'CITY': 'Springfield', 'COUNTY': 'Hampden County', 'STATE': 'Massachusetts'}


In [724]:
missing_conditions = list(db.conditions.find({"patient": {"$in": list(missing_patients)}}))
print(len(missing_conditions))

0


In [731]:
pipeline_12 = list(db['patients_clean'].aggregate([
    {"$lookup": {
        "from": "conditions",
        "localField": "_id",
        "foreignField": "patient",
        "as": "patient_conditions"
    }},
    {"$match": {
        "patient_conditions.conditions": {"$in": ["COVID-19"]}
    }},
    {"$group": {
        "_id": "$RACE",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]))

print(pipeline_12)

[{'_id': 'white', 'count': 6442}, {'_id': 'black', 'count': 656}, {'_id': 'asian', 'count': 548}, {'_id': 'native', 'count': 40}, {'_id': 'other', 'count': 7}]
